In [1]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
from rail.raruma import plotting_functions as raruma_plot
from rail.raruma import utility_functions as raruma_util


In [2]:
from rail.estimation.algos.k_nearneigh import KNearNeighInformer, KNearNeighEstimator
from rail.core.data import Hdf5Handle, DataStore, DATA_STORE
from rail.utils.catalog_utils import RomanRubinCatalogConfig
DS = DATA_STORE()
DataStore.allow_overwrite = True
RomanRubinCatalogConfig.apply(RomanRubinCatalogConfig.tag)

In [3]:
d = tables_io.read("/Users/echarles/pz/sandbox_data/roman_rubin_9925.hdf5")
d.keys()
train = tables_io.sliceObj(d, slice(0, -1, 10))
test = tables_io.sliceObj(d, slice(1, -1, 10))

In [4]:
dh = DS.read_file('input', Hdf5Handle, "/Users/echarles/pz/sandbox_data/roman_rubin_9925.hdf5")

In [5]:
knn_inform = KNearNeighInformer.make_stage(name='knn_inform')

In [6]:
knn_inform.inform(train)

split into 385331 training and 128444 validation samples
finding best fit sigma and NNeigh...



best fit values are sigma=0.017222222222222222 and numneigh=7



Inserting handle into data store.  model_knn_inform: inprogress_model_knn_inform.pkl, knn_inform


In [7]:
knn_inform.finalize()

NOTE/WARNING: Expected output file model_knn_inform.pkl was not generated.


In [8]:
knn_estimate = KNearNeighEstimator.make_stage(name='knn_estimate', connections=dict(model=knn_inform.io.model))

In [9]:
knn_estimate.estimate(test)

Process 0 running estimator on chunk 0 - 10,000
Process 0 estimating PZ PDF for rows 0 - 10,000


AttributeError: 'KNearNeighEstimator' object has no attribute 'kdtree'

In [ ]:
train_targets, train_features = raruma_util.prepare_data_total_mag_and_colors(train, 'LSST_obs_{band}', 'ugrizy')
test_targets, test_features = raruma_util.prepare_data_total_mag_and_colors(test, 'LSST_obs_{band}', 'ugrizy')

In [ ]:
_ = raruma_plot.plot_true_nz(train_targets)

In [ ]:
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [ ]:
scaler = preprocessing.StandardScaler().fit(train_features)
pca = PCA(n_components=train_features.shape[-1], whiten=False)

In [ ]:
scaled_train = scaler.transform(train_features).clip(-5, 5)
pca_train = pca.fit(scaled_train)

In [ ]:
scaled_test = scaler.transform(test_features).clip(-5, 5)
pca_test = pca.fit(scaled_test)

In [ ]:
from sklearn.ensemble import (HistGradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import HuberRegressor, LinearRegression, QuantileRegressor
from sklearn.svm import NuSVR
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor

In [ ]:
hbr = HistGradientBoostingRegressor()
etr = ExtraTreesRegressor()
abr = AdaBoostRegressor()
gpr = GaussianProcessRegressor()
isr = IsotonicRegression()
hur = HuberRegressor()
lir = LinearRegression()
qur = QuantileRegressor()
nsr = NuSVR()
knr = KNeighborsRegressor()
rnr = RadiusNeighborsRegressor()

In [ ]:
def run_it(reg):
    preds = raruma_util.run_regression(reg, scaled_train, train_targets, scaled_test)
    _ = raruma_plot.plot_true_predict_fancy(test_targets, np.nan_to_num(preds, -0.4))
    _ = raruma_plot.plot_biweight_stats_v_redshift(test_targets, preds)

In [ ]:
run_it(hbr)

In [ ]:
run_it(knr)